In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch, torchvision
torch.__version__

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import os
import numpy as np
import json

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/xview/mask_rcnn_R_50_FPN_3x.yaml")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)

In [ ]:
%matplotlib inline 
import cv2
import random
import glob
import matplotlib.pyplot as plt

image_filenames = glob.glob("data/train/images/*")

In [ ]:
input_images = []
output_images = []

from detectron2.utils.visualizer import ColorMode
for d in random.sample(image_filenames, 4):
    im = cv2.imread(d)
    input_images.append(im)
    # TODO: check if need to convert to different color format
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata={}, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    output_images.append(v.get_image()[:, :, ::-1])
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
current_image = output_images[0]
for output_image in output_images[1:]:
    current_image = np.vstack([current_image, output_image])
cv2.imwrite("inference_output.png", current_image)